In [2]:
# Enhanced demo: step-by-step trace + save to file

import sys, pickle, os, time
sys.path.append('../src')

from hmm_model import HangmanHMM
from hangman_env import HangmanEnv
from rl_agent import QLearningAgent
from utils import load_words

# Load trained HMM
with open('../models/hmm_model.pkl', 'rb') as f:
    hmm = pickle.load(f)

# Prepare environment and agent
words = load_words('../data/corpus.txt')
env = HangmanEnv(words, max_wrong_guesses=6)
agent = QLearningAgent(hmm)

# Start a new game (or set a fixed target for reproducibility)
state = env.reset()
target = env.target_word

lines = []
def log(s):
    print(s)
    lines.append(s)

log("="*70)
log("HANGMAN DEMO - STEP BY STEP")
log("="*70)
log(f"Target length: {len(target)}  (word will be revealed at the end)")

step = 0
while not state['game_over'] and step < 50:
    step += 1
    masked = state['masked_word']
    guessed = ''.join(sorted(list(state['guessed_letters'])))
    rem = state['remaining_guesses']

    # HMM probabilities
    probs = hmm.predict_probabilities(masked, state['guessed_letters'])
    topk = sorted(probs.items(), key=lambda x: x[1], reverse=True)[:8]
    topk_str = '  '.join([f"{l}:{p:.3f}" for l, p in topk])

    # Choose action
    action = agent.choose_action(state, probs)

    log(f"\n[Step {step}]")
    log(f"Masked: {masked}   Guessed: {{{guessed}}}   Remaining: {rem}")
    log(f"HMM top-8: {topk_str}")
    log(f"Agent action: {action}")

    # Apply action
    prev_wrong = state['wrong_guesses']
    state, reward, done, info = env.step(action)
    outcome = "REPEAT" if info.get('repeated') else ("HIT" if action in env.target_word else "MISS")
    wrong_inc = state['wrong_guesses'] - prev_wrong
    log(f"Outcome: {outcome}   Reward: {reward}   Wrong now: {state['wrong_guesses']} (+{wrong_inc})")

# Final result
log("\n" + "="*70)
log("GAME OVER")
log(f"Revealed word: {target}")
log(f"Final masked:  {state['masked_word']}")
log(f"Won: {state['won']}  Wrong guesses: {state['wrong_guesses']}")
log("="*70)

# Save trace for submission
os.makedirs('../results', exist_ok=True)
with open('../results/demo_trace.txt', 'w', encoding='utf-8') as f:
    f.write('\n'.join(lines))
print("\n✓ Demo trace saved to ../results/demo_trace.txt")


Loaded 50000 words, 49979 alphabetic
HANGMAN DEMO - STEP BY STEP
Target length: 8  (word will be revealed at the end)

[Step 1]
Masked: ________   Guessed: {}   Remaining: 6
HMM top-8: E:0.109  A:0.093  I:0.084  R:0.072  O:0.069  N:0.066  T:0.062  L:0.060
Agent action: E
Outcome: MISS   Reward: -10   Wrong now: 1 (+1)

[Step 2]
Masked: ________   Guessed: {E}   Remaining: 5
HMM top-8: A:0.116  I:0.109  O:0.093  N:0.072  R:0.064  L:0.063  T:0.062  S:0.060
Agent action: A
Outcome: HIT   Reward: 10   Wrong now: 1 (+0)

[Step 3]
Masked: _A___A__   Guessed: {AE}   Remaining: 5
HMM top-8: N:0.107  L:0.107  I:0.077  R:0.077  D:0.073  S:0.064  K:0.056  T:0.051
Agent action: N
Outcome: HIT   Reward: 10   Wrong now: 1 (+0)

[Step 4]
Masked: _ANN_A__   Guessed: {AEN}   Remaining: 5
HMM top-8: S:0.500  Y:0.250  I:0.250
Agent action: S
Outcome: HIT   Reward: 10   Wrong now: 1 (+0)

[Step 5]
Masked: SANN_AS_   Guessed: {AENS}   Remaining: 5
HMM top-8: Y:0.500  I:0.500
Agent action: Y
Outcome: HIT   